# Explore Epic Details

This notebook is a focused tool for fetching and displaying all available details for a single instrument using its `epic` identifier. 

### How to Use

1.  **Configure the Epic:** In the first code cell, change the value of the `EPIC_TO_QUERY` variable to the epic you want to investigate (e.g., `"OIL_CRUDE"`, `"TSLA"`, `"EURUSD"`).
2.  **Run All Cells:** Execute all cells in the notebook (`Kernel > Restart & Run All`).

The output will be a detailed table containing all instrument properties, dealing rules, and a current market snapshot.

### 1. Setup and Imports

This cell imports the necessary libraries and configures the environment to access our project's source code.

In [ ]:
import os
import sys
from pathlib import Path

import pandas as pd
from loguru import logger

# Add the project root to the Python path to allow module imports
notebook_dir = Path(os.getcwd())
project_root = notebook_dir.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.capitalcom_bot.client_factory import get_client  # noqa: E402

# Configure logger and pandas for clean output
logger.remove()
logger.add(sys.stderr, level="INFO")
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

### 2. Configuration

Define the epic identifier for the instrument you want to investigate.

In [ ]:
# <-- CHANGE THIS VALUE to the epic you want to explore
EPIC_TO_QUERY = "US500"

### 3. Fetch and Display Data

This cell establishes a secure connection using the context manager, calls the API to fetch the detailed data, and formats the response into a readable table.

In [ ]:
if not EPIC_TO_QUERY:
    logger.error(
        "The 'EPIC_TO_QUERY' variable is not set. Please define an epic in the cell above."
    )
else:
    try:
        # The 'with' statement ensures the client is created and the session is closed automatically.
        with get_client(demo_mode=True) as client:
            logger.info(f"--- Fetching full details for {EPIC_TO_QUERY} ---")

            # 1. Fetch the data using the dedicated client method
            full_details = client.get_full_market_details(epic=EPIC_TO_QUERY)

            # 2. Convert the Pydantic model to a dictionary
            details_dict = full_details.model_dump()

            # 3. Normalize the nested dictionary into a flat structure and transpose for readability
            series_details = pd.json_normalize(details_dict).T
            series_details.columns = ["Value"]

            logger.success(
                f"Successfully retrieved details for '{full_details.instrument.name}'."
            )
            display(series_details)

    except Exception as e:
        logger.error(
            f"An error occurred during the API workflow for {EPIC_TO_QUERY}: {e}",
            exc_info=True,
        )